# Test Frame Sender

This file is used for sending individual frames to the device. It measures the accuracy based on individual frames.

**Note that the software on the MCU must match this, and not classify based on clusters of frames.**

In [1]:
import numpy as np
import pandas as pd
import gc

In [2]:
df = pd.read_csv("./balanced_dataset.csv", header=None)

In [3]:
dataset = df.iloc[:, :-1].to_numpy(dtype=np.float32)            # All but last column as float32
labels_set = df.iloc[:, -1].to_numpy(dtype=str)                 # Last column as string
del df
gc.collect()

0

In [4]:
dataset = dataset.reshape(dataset.shape[0], 16, 8, 1)
input_shape = dataset[0].shape

print(f"Dataset shape: {dataset.shape}")
print(f"Labels shape: {labels_set.shape}")
print(f"Input shape: {input_shape}")

Dataset shape: (776566, 16, 8, 1)
Labels shape: (776566,)
Input shape: (16, 8, 1)


In [5]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(dataset, labels_set, test_size=0.05, random_state=42, stratify=labels_set)
del dataset, labels_set
gc.collect()

0

In [6]:
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

# Hot end code the labels.
label_encoder = LabelEncoder()
y_train = to_categorical(label_encoder.fit_transform(y_train))
y_test = to_categorical(label_encoder.fit_transform(y_test))

2025-04-10 10:40:20.757770: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-04-10 10:40:20.826195: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-04-10 10:40:20.873920: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744274420.922463   10668 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744274420.938231   10668 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1744274421.036024   10668 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linkin

In [7]:
def indexOfMax(arr):
    return np.argmax(arr)

In [52]:
import serial
print("Configuring serial port...")
ser = serial.Serial(
    port='/dev/ttyACM0',  # Change this to your actual port, e.g., 'COM3' on Windows, '/dev/ttyS0' on Linux
    baudrate=115200,       # Set baud rate to 115200
    bytesize=serial.EIGHTBITS,
    parity=serial.PARITY_NONE,
    stopbits=serial.STOPBITS_ONE,
    timeout=1              # Set timeout for reading
)

Configuring serial port...


In [44]:
if ser.is_open:
    print(f"Serial port {ser.port} opened at {ser.baudrate} baud.")

Serial port /dev/ttyACM0 opened at 115200 baud.


In [45]:
def calc_accuracy(y_true, y_pred):
    success = 0
    failure = 0
    for i in range(len(y_pred)):
        if int(y_pred[i]) == int(indexOfMax(y_true[i])):
            success += 1
        else:
            failure += 1
    res = round(success / (success + failure), 4)
    return res

In [46]:
import struct

x = x_test
y = y_test[0:len(x)]
print(x.shape)

results = []
total = len(x)

(38829, 16, 8, 1)


In [47]:

# Cast y[0] into int arrat
y[0].astype(int)

array([0, 0, 0, 1])

In [53]:


# Send data
for i in range(total):
    data = x[i].flatten()
    label = y[i]
    if np.argmax(label) != 0:
        #print(f"Skipping {i} - {label.astype(int)}")
        continue
    for j in range(len(data)):
        data_to_send = struct.pack('f', data[j])
        ser.write(data_to_send)
    while ser.in_waiting > 0:
        response = ser.readline().decode('utf-8').strip()
        if response:
            print(f"Received {response} but is {y[i].astype(int)}")
    # No need for sleep here as the delay is already managed by the serial communication timeout.
    # Read response
    #print(f"Guess: {response} - Class: {indexOfMax(y[i])} Accuracy: {calc_accuracy(y, results)} ({i}/{total})")


Received s:Heap: 1100 / 461228 bytes but is [1 0 0 0]
Received s:Tensor arena size: 36164/37000 bytes but is [1 0 0 0]
Received s:Input tensor shape: 1, 16, 8, 1 but is [1 0 0 0]
Received s:Output tensor shape: 1, 4 but is [1 0 0 0]
Received Ready to start but is [1 0 0 0]
Received c: [0,1,0,0] but is [1 0 0 0]
Received c: [0,2,0,0] but is [1 0 0 0]
Received c: [0,3,0,0] but is [1 0 0 0]
Received c: [0,4,0,0] but is [1 0 0 0]
Received c: [0,5,0,0] but is [1 0 0 0]
Received c: [0,6,0,0] but is [1 0 0 0]
Received c: [0,7,0,0] but is [1 0 0 0]
Received c: [1,7,0,0] but is [1 0 0 0]
Received c: [1,8,0,0] but is [1 0 0 0]
Received c: [1,9,0,0] but is [1 0 0 0]
Received c: [1,10,0,0] but is [1 0 0 0]
Received c: [1,11,0,0] but is [1 0 0 0]
Received c: [1,12,0,0] but is [1 0 0 0]
Received c: [1,13,0,0] but is [1 0 0 0]
Received c: [1,14,0,0] but is [1 0 0 0]
Received c: [1,15,0,0] but is [1 0 0 0]
Received c: [1,16,0,0] but is [1 0 0 0]
Received c: [1,17,0,0] but is [1 0 0 0]
Received c: [1,1

KeyboardInterrupt: 

In [54]:
# Close the serial port
ser.close()